In [ ]:
!module purge
!module load apps/anaconda3
!module load compilers/nvidia/cuda/11.2
!module load compilers/gcc/8.3.0

In [1]:

import cv2
import os
import random

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
import tensorflow as tf
import tensorflow.keras as keras


In [4]:
tf.__version__

'2.8.0'

In [5]:
tf.random.set_seed(42)
np.random.seed(42)


In [6]:
train_dir = "/home/191112024/splited_dataset/AGCCPF/train"
test_dir = "/home/191112024/splited_dataset/AGCCPF/test"
valid_dir = "/home/191112024/splited_dataset/AGCCPF/validate"


In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
# !unzip AGCCPF.zip

In [10]:

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',
        shuffle=True
        )

valid_generator = test_datagen.flow_from_directory(
        valid_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',
        shuffle=True
        )

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical',
        shuffle=False
        )

Found 5253 images belonging to 3 classes.
Found 1315 images belonging to 3 classes.
Found 1644 images belonging to 3 classes.


In [11]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [12]:
from tensorflow.keras import optimizers

In [17]:
input_layer=layers.Input((224, 224, 3))
from tensorflow.keras.applications import VGG16,DenseNet121
conv_base_1 = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))(input_layer)


# conv_base = EfficientNetB7(weights='imagenet',
#                   include_top=False,
#                   input_shape=(200, 200, 3))

conv_base_2 = DenseNet121(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))(input_layer)

# conv_base = MobileNetV2(weights='imagenet',
#                   include_top=False,
#                   input_shape=(224, 224, 3))

In [19]:
# model_1 = keras.models.Sequential()
model_1=conv_base_1
model_1=keras.layers.GlobalAveragePooling2D()(model_1)
model_1=keras.layers.Dense(3, activation='softmax')(model_1)

# model_2 = keras.models.Sequential()
model_2=conv_base_2
model_2=keras.layers.GlobalAveragePooling2D()(model_2)
model_2=keras.layers.Dense(3, activation='softmax')(model_2)

merge=layers.Concatenate()([model_1,model_2])
result=layers.Dense(3, activation='softmax')(merge)

model=keras.models.Model(input_layer,result)



In [20]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu',
#                         input_shape=(150, 150, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(512, activation='relu'))
# model.add(layers.Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc',tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [21]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "MobileNet_model.h5", save_best_only=True)
earlystop_cb = keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True)


In [22]:
model.summary(0)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, 7, 7, 512)    14714688    ['input_3[0][0]']                
                                                                                                  
 densenet121 (Functional)       (None, 7, 7, 1024)   7037504     ['input_3[0][0]']                
                                                                                                  
 global_average_pooling2d_4 (Gl  (None, 512)         0           ['vgg16[0][0]']            

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=260,
                    epochs=50,
                    validation_data=valid_generator,
                    validation_steps=60,
                    callbacks=[checkpoint_cb])


Epoch 1/50
 65/260 [======>.......................] - ETA: 14:37 - loss: 1.1103 - acc: 0.3338 - recall: 0.0323 - precision: 0.3652

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10, 10))
plt.ylim((0,2))
plt.grid(True)
plt.show()

In [ ]:
model.evaluate(test_generator)

In [ ]:
x=model.predict(test_generator)

In [ ]:
pred=np.argmax(x, axis=-1)
pred

In [ ]:
actual=test_generator.labels
actual

In [ ]:
pred=np.argmax(x, axis=-1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(actual,pred)

In [ ]:
x=model.predict(test_generator)
actual=test_generator.labels
pred=np.argmax(x, axis=-1)
from sklearn.metrics import confusion_matrix
confusion_matrix(actual,pred)